# Konvertering av N50 Vektor til GeoParquet

## 1. Last ned N50 Kartdata

![Geonorge N50](img/geonorge-n50.png)

Vektordata kan fritt lastes ned fra kartkatalogen.geonorge.no, du finner N50 [Her](https://kartkatalog.geonorge.no/metadata/n50-kartdata/ea192681-d039-42ec-b1bc-f3ce04c189ac)

Til denne demoen brukes:
- `Hele landet`
- `UTM 33`
- `FGBD`

**OBS!! Hele landet er et relativt stort datasett, det er også fint mulig å bruke et mindre datasett.**

![Geonorge Last ned](img/geonorge-lastned.png)

Du får en zip fil når du laster ned, pakk ut denne i `src/geoparquet`. Git er satt opp til å ikke tracke `.gdb` mapper.

In [ ]:
# import packages
import fiona
import geopandas as gpd
from pathlib import Path

import utils

## Data input

Starter med å lese ut alle lagene som finnes i `.gdb` filen.

In [ ]:
# Leser alle lagene fra.gdb filen
path = utils.get_workdir() / Path("Basisdata_0000_Norge_25833_N50Kartdata_FGDB.gdb")
layers = fiona.listlayers(path)
print(layers)

## Parquet produksjon

For hvert lag produseres en parquet fil i `/out` mappen.

In [ ]:
# Oppretter /out mappen
utils.create_dir("out")
# Leser hvert lag fra .gdb filen og skriver til geoparquet
for layer in layers:
    try:
        n50_df = gpd.read_file(path, layer=layer)
        n50_df.to_parquet(
            path=f"out/{layer}.snappy.parquet",
            compression="snappy",
            geometry_encoding="WKB",
            write_covering_bbox=True,
        )
    except Exception as e:
        print(f"Failed to load layer: {layer} with error: {e}")

## Setter fil og bbox

In [ ]:
file_path = 'out/N50_Samferdsel_senterlinje.snappy.parquet'
# file_path = 'out/N50_Arealdekke_omrade.snappy.parquet'
xmin = 82100
xmax = 96000
ymin = 6461800
ymax = 6470500

## Initialiserer en DuckDB database i minne

Credit: https://github.com/Norkart/syntaks-2025/blob/main/parquet-create.ipynb

In [ ]:
import duckdb

# Connect to DuckDB
con = duckdb.connect(database=':memory:')

# Load the necessary extensions
con.execute("INSTALL spatial;")
con.execute("LOAD spatial;")

con.execute("INSTALL httpfs;")
con.execute("LOAD httpfs;")



## Leser ut metadata

In [ ]:
query = f"""
SELECT *
FROM parquet_metadata('{file_path}')
"""
result = con.query_geo(query).to_df()
result

## Leser schema

In [ ]:

query = f"""
SELECT *
FROM parquet_schema('{file_path}')
"""
result = con.query_geo(query).to_df()
result

## Leser vektordata

In [ ]:
query = f"""
SELECT *
FROM read_parquet('{file_path}')
LIMIT 10
"""
result = con.query_geo(query).to_df()
result

## Leser vektor data i bounding box

In [ ]:
query = f"""
SELECT *
FROM read_parquet('{file_path}')
WHERE bbox.xmin BETWEEN {xmin} AND {xmax}
AND bbox.ymin BETWEEN {ymin} AND {ymax}
LIMIT 10;
"""
result = con.query_geo(query).to_df()
result

## Simulerer netverksfil med å hoste i docker via nginx

```bash
# from src/geoparquet/
docker build -t out-nginx .
docker run --name s3-nginx -d -p 8000:80 out-nginx:latest
```

In [ ]:

query = f"""
SELECT *
FROM read_parquet('http://localhost:8000/N50_Samferdsel_senterlinje.snappy.parquet')
WHERE bbox.xmin BETWEEN {xmin} AND {xmax}
AND bbox.ymin BETWEEN {ymin} AND {ymax}
LIMIT 10
"""
result = con.query_geo(query).to_df()
result

## Viser data på interaktivt kart

In [ ]:
kristiansand = gpd.read_parquet(
    path=file_path,
    bbox=(xmin, ymin, xmax, ymax)
)
kristiansand.explore()